In [9]:
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import PIL
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox
import glob
from moviepy.editor import *
import cv2
import math
from PIL import Image
import pandas as pd

In [12]:
#path the video
vidcap = cv2.VideoCapture(r"D:\Video_data\train\fire\fire0.mp4")
# success, image = vidcap.read()
count = 0
while True:
    success, image = vidcap.read()
#     if success == False:
#         break;
    cv2.imwrite(r"D:\frames\fire0\%d.png" % count, image)
    count+=1

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [7]:
i=0
Imagee = []
xCoord = []
yCoord = []
dat = []
dct = {}
effectiveArea = 0
area = []
hei = []
wid = []
Area =[]
for i in range(3):

    image_for_testing='fire0'+"/"+str(i)+".png"
    
    
    img = cv2.imread(str(image_for_testing))

    # convert to grayscale
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    lower = np.array([0, 40, 200])
    upper = np.array([35, 215, 255])
    mask = cv2.inRange(hsv, lower, upper)
    result = img.copy()
    blur= cv2.GaussianBlur(mask,(7,7),0)
    contours = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        if(w*h > 100 ):
            cv2.rectangle(result, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        xCoord = xCoord + [x]
        yCoord = yCoord + [y]
        hei = hei + [h]
        wid = wid + [w]
        
        effectiveArea = effectiveArea + w*h
        for t in range(len(xCoord)-1):
            area = area + [w*h]
        
        rects = []
        rectsUsed = []
        
        for cnt in contours:
            rects.append(cv2.boundingRect(cnt))
            rectsUsed.append(False)
            
        def getXFromRect(item):
            return item[0]
        
        rects.sort(key = getXFromRect)
        
        acceptedRects = []
        xThr = 5
        
        for supIdx, supVal in enumerate(rects):
            if (rectsUsed[supIdx] == False):
                currxMin = supVal[0]
                currxMax = supVal[0] + supVal[2]
                curryMin = supVal[1]
                curryMax = supVal[1] + supVal[3]
                rectsUsed[supIdx] = True
                
                for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):
                    candxMin = subVal[0]
                    candxMax = subVal[0] + subVal[2]
                    candyMin = subVal[1]
                    candyMax = subVal[1] + subVal[3]
                    if (candxMin <= currxMax + xThr):
                        currxMax = candxMax
                        curryMin = min(curryMin, candyMin)
                        curryMax = max(curryMax, candyMax)
                        rectsUsed[subIdx] = True
                    else:
                        break
                acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])
                
        for rect in acceptedRects:
            img = cv2.rectangle(image, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
            
        for k in range(len(xCoord)-1):
            dct[k] = {"X-cord":xCoord[k],"Y-cord":yCoord[k],"Height":hei[k],"Width":wid[k],"Area":area[k]}
            dat = dat + [dct[k]]
            dataa = pd.DataFrame(dat)
        
        dataa.to_excel("outputData.xlsx")
        cv2.imwrite('fire0'+str(i)+'.jpg',result)
        imgee = PIL.Image.open('fire0'+str(i)+'.jpg')
        width = imgee.width
        height = imgee.height
        print(str(effectiveArea)+ " / " + str(width*height))     # FOR CALCULATNG TOTAL FIRED AREA AND TOTAL AREA OF IMAGE
        print(str(((effectiveArea)/(width*height))*100) + "% area is Fired")
        
        Imagee = Imagee + ['fire0'+str(i)+'.jpg']
  
        cv2.imshow("image", result)
        cv2.waitKey(0)








error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
